## Proposal_Differentiation_Tool

Differentiate between Titles, Headers, Body paragraphs, etc.

### Initialize

In [1]:
import fitz
import json
from re import findall

### Get All Text tool

In [34]:
# Select document & page number (0-indexed) to read
doc = fitz.open("./downloaded/Bike.pdf")
page_num = 12

# Check if Table of Contents (bookmarks) exists for the PDF
toc = doc.getToC()
if len(toc) != 0:
    # Use ToC to differentiate
    pass

# Run function to get the plain text
get_pdf_text(doc, page_num)



Ulteig 9

Will Kerns, AICP 

♦ 

Project Manager

Will is a Transportation Planning and Engineering expert, with 10+ years leading multimodal transportation planning projects. Certified through American Institute of Certified Planners (AICP). Proficient in stakeholder outreach, technical writing, GIS mapping and coalition building. Will is a main street urban designer focused on assisting communities become better connected, more economically competitive, and more inclusive of diverse populations. Will’s experience is in infrastructure project planning, directing mechanical engineering projects, leading diverse teams, and integrating public health into the planning process. Will is knowledgeable in NEPA projects, corridor studies, feasibility studies, funding procurement, and healthy community design. Will also works with Hyperloop Transportation Technologies as their Global Planning and Infrastructure Lead, and Will’s team at HyperloopTT recently constructed a full-scale Hyperloop t

'as a local liaison and will assist the team when appropriate. He is also a Brandon Valley High alum and is eager to help his former City!'

In [33]:
def get_pdf_text(doc, page_num):
    # Get page's text (json format)
    json_text = doc[page_num].getText("json")

    # Group json entries by those that share the same color&font&size

    # Convert string/json to usable format (dictionary/lists)
    json_text = json.loads(json_text)  
    text_blocks = json_text["blocks"]
    
    # Set previous properties to None to prepare for comparisons
    prev_color = None
    prev_font = None
    prev_size = None
    text_holder = ""

    # Iterate through all the text entries
    for entry in text_blocks:
        # Check if this block is an image
        try:
            img = entry["image"]
            print("\n[IMAGE]\n")    # <--- For testing. Remove later. Images should be ignored, esp. because they can ruin text
        except:
            # Not an image
            pass

        # Check if this block exists, or if the end of the page has been reached
        try: 
            lines = entry["lines"]
        except:
            # *** Will need to change this - only print if it hadn't already been printed ***           
            # (check that last entry is different from what is currently in holder)
            print(text_holder)
            break

        # Line is not an image and contains text so we continue
        if lines != None:
            for spans in lines:
                # Get the line data
                spans = spans["spans"]                

                for i in range(len(spans)):
                    span = spans[i]
                    
                    # Identify current line's properties and text
                    font = span["font"]
                    color = span["color"]
                    size = span["size"]
                    text = span["text"]

                    # Check if current line has different properties from previous
                    if (font != prev_font) or (color != prev_color) or (size != prev_size):
                        # Current line has different properties.

                        # For now, just print whatever text had the previous properties
                        print(text_holder)
                        print()

                        # Reset text holder & prev_ properties
                        text_holder = text
                        prev_font = font
                        prev_size = size
                        prev_color = color

                    else:
                        # Current line has the same properties. Add it to the text block.
                        text_holder += text
                        
    return text_holder

### Differentiation between Text and Headers

In [60]:
# Select document & page number (0-indexed) to read
doc = fitz.open("./downloaded/Bike.pdf")
page_num = 12

# Check if Table of Contents (bookmarks) exists for the PDF
toc = doc.getToC()
if len(toc) != 0:
    # Use ToC to differentiate
    pass

# Run function to identify what properties are plain text
ct_by_props = differentiate_pdf_text(doc, page_num)
for key in ct_by_props:
    print(key, "==", ct_by_props[key])

# Select body text by whatever set of text props has the largest wordcount
body_text_props = find_body_text(ct_by_props)


('NewsGothicStd', 9.0, 13027530) 
 
0


('NewsGothicStd', 9.0, 13027530) 
 Ulteig 
1


('NewsGothicStd-BoldObliqu', 10.100000381469727, 32117) 
 9
1


('SymbolMT', 10.100000381469727, 32117) 
 Will Kerns, AICP 
3


('NewsGothicStd-BoldObliqu', 10.100000381469727, 12291437) 
 ♦ 
0


('ArialNarrow', 10.100000381469727, 5000266) 
 Project Manager
2


('ArialNarrow', 10.100000381469727, 5000266) 
 Will is a Transportation Planning and Engineering 
7


('ArialNarrow', 10.100000381469727, 5000266) 
 expert, with 10+ years leading multimodal 
6


('ArialNarrow', 10.100000381469727, 5000266) 
 transportation planning projects. Certified through 
5


('ArialNarrow', 10.100000381469727, 5000266) 
 American Institute of Certified Planners (AICP). 
6


('ArialNarrow', 10.100000381469727, 5000266) 
 Proficient in stakeholder outreach, technical 
5


('ArialNarrow', 10.100000381469727, 5000266) 
 writing, GIS mapping and coalition building. Will is a main street 
11


('ArialNarrow', 10.10000038146

In [51]:
def count_words(s):
    return len(findall(r'\w+', s))

# Group json entries by those that share the same color&font&size given a document and page number
def differentiate_pdf_text(doc, page_num):
    # Get page's text (json format)
    json_text = doc[page_num].getText("json")

    # Convert string/json to usable format (dictionary/lists)
    json_text = json.loads(json_text)  
    text_blocks = json_text["blocks"]
    
    # Set previous properties to None to prepare for comparisons
    prev_props = (None, None, None)
    text_holder = ""
    
    # Create a dictionary to hold word counts for each set of properties - props:ct
    ct_by_props = {}

    # Iterate through all the text entries to count words for each set of properties
    for entry in text_blocks:
        # Check if this block is an image
        try:
            img = entry["image"]
        except:
            pass

        # Check if this block exists, or if the end of the page has been reached
        try: 
            lines = entry["lines"]
        except:
            break

        # Line is not an image and contains text so we continue
        if lines != None:
            for spans in lines:
                # Get the line data
                spans = spans["spans"]                

                for i in range(len(spans)):
                    span = spans[i]
                    
                    # Identify current line's properties and text
                    # put into tuple - (font, size, color)
                    props = (span["font"], span["size"], span["color"])
                    text = span["text"]

                    ct = count_words(text_holder)
                    
                    # DEBUG: printing 
                    print("\n=====================")
                    print(props, "\n", text_holder)
                    print(ct)
                    print("=====================\n")

                    # Update dictionary
                    if props not in ct_by_props:
                        ct_by_props[props] = ct
                    else:
                        ct_by_props[props] = ct_by_props[props] + ct

                    # Update text holder & prev properties to new
                    text_holder = text
                    prev_props = props

                        
    #return print("(Font, Size, Color) =", paragraph_props)
    return ct_by_props

In [55]:
def find_body_text(ct_by_props):
    return

### Notes section (Misc. code used for debugging)

In [26]:
# Check what style is like for json entries/spans/span/text
print_first_entry(doc,page_num)

LINES:
 [{'spans': [{'size': 9.0, 'flags': 4, 'font': 'NewsGothicStd', 'color': 13027530, 'ascender': 1.0199999809265137, 'descender': -0.25, 'text': 'CITY OF BRANDON BIKE & PEDESTRIAN PLAN', 'origin': [36.52000045776367, 763.0020141601562], 'bbox': [36.52000045776367, 753.822021484375, 243.70001220703125, 765.2520141601562]}], 'wmode': 0, 'dir': [1.0, 0.0], 'bbox': [36.52000045776367, 753.822021484375, 243.70001220703125, 765.2520141601562]}, {'spans': [{'size': 9.0, 'flags': 4, 'font': 'NewsGothicStd', 'color': 13027530, 'ascender': 1.0199999809265137, 'descender': -0.25, 'text': '10', 'origin': [491.2922058105469, 763.0020141601562], 'bbox': [491.2922058105469, 753.822021484375, 502.5602111816406, 765.2520141601562]}], 'wmode': 0, 'dir': [1.0, 0.0], 'bbox': [491.2922058105469, 753.822021484375, 502.5602111816406, 765.2520141601562]}] 

SPANS:
 [{'size': 9.0, 'flags': 4, 'font': 'NewsGothicStd', 'color': 13027530, 'ascender': 1.0199999809265137, 'descender': -0.25, 'text': 'CITY OF B

In [25]:
def print_first_entry(doc, page_num):
    # Get page's text (json format)
    json_text = doc[page_num].getText("json")

    # Convert string/json to usable format (dictionary/lists)
    json_text = json.loads(json_text)  
    text_blocks = json_text["blocks"]
    
    # Iterate through all the text entries
    for entry in text_blocks:
        
        lines = entry["lines"]
        print("LINES:\n", lines, "\n")

        if lines != None:
            for spans in lines:       
                # Get the line data
                spans = spans["spans"]
                
                print("SPANS:\n", spans, "\n")

                for i in range(len(spans)):
                    span = spans[i]
                    
                    print("SPAN:\n", span, "\n")
                    
                    # Identify current line's properties and text
                    text = span["text"]
                    print("TEXT:\n", text, "\n")
                    
        break

    return

In [21]:
def get_json_text(doc, page_num):
    # Get page's text (json format)
    json_text = doc[page_num].getText("json")

    # Group json entries by those that share the same color&font&size

    # Convert string/json to usable format (dictionary/lists)
    json_text = json.loads(json_text)  
    text_blocks = json_text["blocks"]

    print(text_blocks)
    return text_blocks    